In [ ]:
#한겨레
from bs4 import BeautifulSoup
import requests
import re
import urllib.parse
from datetime import timedelta
import datetime, re

def hani_newsurl(keyword, date1, date2, pagenum):
    url=("http://search.hani.co.kr/Search?command=query&keyword={0}" \
"&media=news&sort=d&period=all&datefrom={1}" \
"&dateto={2}" \
"&pageseq={3}"
    .format(keyword,date1,date2,pagenum))
    resp = requests.get(url)
    resp.raise_for_status()
    resp.encoding = 'utf-8'
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')
    my_titles = str(soup.select('dt,a'))
    news_url = re.findall('href="(http:.+?)"', my_titles)
    return (news_url)

date1=datetime.date(2018, 10, 1)
date2=datetime.date(2018, 12, 31)
keyword='유치원3법'

url_list = set()
while date1 < date2:
    print(date1)
    for num in range(20):
        t = hani_newsurl(keyword, re.sub('-','.',str(date1)), re.sub('-','.',str(date2)), num)
        url = [i for i in t if re.search('hani',i)]
        for i in url:
            url_list.add(i)
        date1+=timedelta(days=+1)
        
def hani_news(url):
    source_code_from_url = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding = 'utf-8')
    text = str(soup.select('#a-left-scroll-in > div.article-text > div > div.text'))
    burn = re.compile('\t|\n|\r|\'|\[|\]|\{.+|<.+?>')
    article = burn.sub('',text)
    article = re.sub('\.','\n',article)
    press = 'hani'
    date = str(soup.select('#article_view_headline > p.date-time'))
    return (article, press, date) 

import pandas as pd 

article = []
press = []
date = []
url = []

for i in url_list:
    if hani_news(i)[0]!='':
        article.append(hani_news(i)[0])
        press.append(hani_news(i)[1])
        date.append(hani_news(i)[2])
        url.append(i)   
        
new_date=[]
for i in date:
    if re.search('\[<p class="date-time"><span><em>등록 :</em>',i):
        p = re.compile('\[<p class="date-time"><span><em>등록 :</em>(.+?)\s')
        new_date+=p.findall(i)
    else:
        new_date+=i
date = [re.sub('-','.',i) for i in new_date]
        
data = {'article': article, 'press': press, 'date': date, 'url': url}
df = pd.DataFrame(data, columns = ['article', 'press', 'date','url'])
df.to_csv(r'C:/Users/chsjk/OneDrive/Documents/학업/4학년 겨울학기/웹스크롤러, 형태소 분석기,코퍼스/article/한겨레_유치원3법.csv')

df[['article', 'press', 'date','url']]